In [1]:
# Reads in an input lidar file and writes an
# output image file with the minimum 'Z' value for
# each bin
import numpy
from numba import jit
from pylidar import lidarprocessor
from pylidar.toolbox import spatial
from pylidar.lidarformats import generic

BINSIZE = 1.0

@jit
def findMinZs(data, outImage, xMin, yMax):
    for i in range(data.shape[0]):
        if data[i]['CLASSIFICATION'] == lidarprocessor.CLASSIFICATION_GROUND:
            row, col = spatial.xyToRowColNumba(data[i]['X'], data[i]['Y'],
                    xMin, yMax, BINSIZE)
            if outImage[row, col] != 0:
                if data[i]['Z'] < outImage[row, col]:
                    outImage[row, col] = data[i]['Z']
            else:
                outImage[row, col] = data[i]['Z']

def processChunk(data, otherArgs):
    lidar = data.input1.getPoints(colNames=['X', 'Y', 'Z', 'CLASSIFICATION'])
    findMinZs(lidar, otherArgs.outImage, otherArgs.xMin, otherArgs.yMax)

inFile = 'building.las'
info = generic.getLidarFileInfo(inFile)
header = info.header

dataFiles = lidarprocessor.DataFiles()
dataFiles.input1 = lidarprocessor.LidarFile(inFile, lidarprocessor.READ)

xMin, yMax, ncols, nrows = spatial.getGridInfoFromHeader(header, BINSIZE)

outImage = numpy.zeros((nrows, ncols))

otherArgs = lidarprocessor.OtherArgs()
otherArgs.outImage = outImage
otherArgs.xMin = xMin
otherArgs.yMax = yMax

lidarprocessor.doProcessing(processChunk, dataFiles, otherArgs=otherArgs)

#outFile = 'raster'
iw = spatial.ImageWriter(outFile, tlx=xMin, tly=yMax, binSize=BINSIZE)
iw.setLayer(outImage)
iw.close()

/Users/cheng/miniconda3/envs/myenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'outFile' is not defined

In [2]:
from pylidar import lidarprocessor
from pylidar.toolbox import spatial
from pylidar.toolbox import interpolation

BINSIZE = 1.0

inFile = 'data/1641_2617.las'
data = spatial.readLidarPoints(inFile,
        classification=lidarprocessor.CLASSIFICATION_GROUND)

(xMin, yMax, ncols, nrows) = spatial.getGridInfoFromData(data['X'], data['Y'],
            BINSIZE)

pxlCoords = spatial.getBlockCoordArrays(xMin, yMax, ncols, nrows, BINSIZE)

dem = interpolation.interpGrid(data['X'], data['Y'], data['Z'], pxlCoords, 'pynn')

#outFile = 'building_dem'
iw = spatial.ImageWriter(outFile, tlx=xMin, tly=yMax, binSize=BINSIZE)
iw.setLayer(dem)
iw.close()

NameError: name 'outFile' is not defined